In [1]:
import pandas as pd
import helium as hell
import requests
import time

In [2]:
df=pd.read_html('https://twiki.cern.ch/twiki/bin/view/AtlasPublic/Publications')

In [3]:
df[0]

,Short Title,Group,Journal reference,Submitted on,√s (TeV),L,Links
0,Lepton Flavor Violating Z Decay with a Leptoni...,EXOT,Submitted to PRL,26-MAY-21,13,139 fb-1,Documents | Internal
1,Performance of ATLAS topological trigger Full ...,TRIG,Submitted to EPJC,04-MAY-21,13,139 fb-1,Documents | 2105.01416 | Internal
2,Search for H(125) --> gamma gamma + Missing-ET...,HIGG,Submitted to JHEP,27-APR-21,13,139 fb-1,Documents | 2104.13240 | Internal
3,Search for stopped long-lived particles decayi...,SUSY,Submitted to JHEP,07-APR-21,13,111 fb-1,Documents | 2104.03050Briefing | Internal
4,Measurement of ttZ cross sections in Run 2Full...,TOPQ,Submitted to EPJC,23-MAR-21,13,139 fb-1,Documents | 2103.12603 | Inspire HepData | Int...
...,...,...,...,...,...,...,...
998,ATLAS inner detector readinessFull Title:Keywo...,IDET,Eur. Phys. J. C 70 (2010) 787,26-APR-10,0.9,0 μb-1,Documents | 1004.5293 | Inspire | Internal
999,Charged-particle multiplicities at 900 GevFull...,STDM,Phys. Lett. B 688 (2010) 21,16-MAR-10,0.9,9 μb-1,Documents | 1003.3124 | Inspire Rivet | HepDat...
1000,LAr drift timeFull Title:Keywords:Liquid-argon...,LARG,Eur. Phys. J. C 70 (2010) 755,22-FEB-10,0.9,0 μb-1,Documents | 1002.4189 | Inspire | Internal
1001,LAr readinessFull Title:Keywords:Liquid-argon ...,LARG,Eur. Phys. J. C 70 (2010) 723,14-DEC-09,0.9,0 μb-1,Documents | 0912.2642 | Inspire | Internal


In [4]:
browser=hell.start_firefox()
browser.get('https://twiki.cern.ch/twiki/bin/view/AtlasPublic/Publications')

In [5]:
t=browser.find_elements_by_tag_name('tbody')
tr=t[0].find_elements_by_tag_name('tr')
k=[h.text.strip() for h in tr[0].find_elements_by_tag_name('th')]

In [12]:
atl=pd.DataFrame()

for i in range(1, len(tr)):
    print(i,end='\r')

    time.sleep(0.5)
    row={}
     
    #get row values
    td=tr[i].find_elements_by_tag_name('td')

    # build row dictionary
    for j in range(len(td)):
        row[k[j]]=td[j].text

    #Add inspire API info
    if row.get('Links').find('Inspire')>-1:
        insp=td[6].find_element_by_link_text('Inspire')
    else:
        atl=atl.append(row,ignore_index=True)
        continue

    l=insp.get_attribute('href').replace('record','api/literature')
    row['inspire_id']=l.split('/')[-1]

    r=requests.get(l)

    if r.status_code!=200:
            atl=atl.append(row,ignore_index=True)
            continue

    try:
        row['title']=r.json().get('metadata').get('titles')[0].get('title')
    except:
        row['abstract']=''    
    try:
        row['abstract']=r.json().get('metadata').get('abstracts')[0].get('value')
    except:
        row['abstract']=''
    try:
        row['keywords']=[d.get('value') for d in r.json().get('metadata').get('keywords') 
                              if d.get('schema')=='INSPIRE' ]
    except:
        row['keywords']=[]
    try:
        row['doi']=r.json().get('metadata').get('dois')[0].get('value')
    except:
        row['doi']=''
    atl=atl.append(row,ignore_index=True)

In [13]:
atl.columns

Index(['Group', 'Journal reference', 'L', 'Links', 'Short Title',
       'Submitted on', '√s (TeV)', 'abstract', 'doi', 'inspire_id', 'keywords',
       'title'],
      dtype='object')

In [16]:
atl['inspire_id'].dropna().shape

(998,)

In [17]:
atl=atl[['Short Title','Group', 'Journal reference','doi', 'Submitted on',  'L', '√s (TeV)', 'Links', 'inspire_id',
      'title', 'keywords', 'abstract' ]]
atl.shape

(1003, 12)

In [21]:
atl=atl.fillna('')

In [22]:
atl.to_json('atlas_pub.json',orient='records')
atl.to_csv('atlas_pub.csv',index=False)

In [25]:
atl[atl['title']!=''].shape

(997, 12)

In [ ]:
1+1